# Índice

- [Sistemas de recomendación](#recommender)
    - [Filtros colaborativos](#cf)
- [Descarga de datos](#descarga)
- [Carga de datos](#carga)
- [Limpieza de datos](#limpieza)
- [Análisis descriptivo](#analisis)
- [Modelo y ajuste de parámetros](#model)
    - [K-fold](#kfold)
- [Resultado del modelo](#resultado)


    
<div id='xx' />

La práctica consiste en construir un recomendador con el módulo de recomendación de Spark (en la versión 1.6 solamente implementa el método ALS) sobre un conjunto de entrenamiento, realizar predicciones sobre un conjunto de test y posteriormente evaluar su rendimiento aplicando un RegressionEvaluator con metricName=”rmse”, dado que los valores finales son las valoraciones que se supone que daría el usuario a una determinada película que aún no ha visto, y por tanto numéricos.

El paquete de pySpark de recomendaciones se encuentra en:

http://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.recommendation.ALS

https://spark.apache.org/docs/latest/ml-collaborative-filtering.html

El recomendador se construirá utilizando el dataset de 100000 puntuaciones de 1000 usuarios sobre 1700 películas de Movielens

http://grouplens.org/datasets/movielens/100k/  (user id | item id | rating | timestamp)

https://grouplens.org/datasets/movielens/

Se recomienda leer el archivo README.txt para entender bien los datos:

http://files.grouplens.org/datasets/movielens/ml-100k-README.txt

Para poder practicar sobre un dataset más pequeño o si alguien tiene problemas de rendimiento se puede utilizar éste más reducido:

https://raw.githubusercontent.com/apache/spark/master/data/mllib/als/sample_movielens_ratings.txt

<div id='recommender' />

## Sistemas de recomendación (Recommender Systems)

Los sistemas de recomendación ayudan a emparejar usuarios con productos. Seleccionan el producto que maximiza el valor, tanto para el comprador como para el vendedor en un momento determinado.

Podemos distinguir distintos tipos:

- Basados en conocimiento experto
- Basados en contenido
- Filtros colaborativos
- Sistemas híbridos
- Dependientes del contexto
- Recomendadores agrupados
- Recomendadores sociales



<div id='cf' />

#### Filtros colaborativos (Collaborative Filtering)

[Documentación Spark](https://spark.apache.org/docs/latest/ml-collaborative-filtering.html)

Se basan en el uso de la sabiduría popular para recomendar elementos, es decir, generan recomendaciones de elementos comparando patrones de comportamiendo de los usuarios. Se fundamentan en la asunción de que los usuarios que tuvieron gustos similares en el pasado los tendrán igualmente en el futuro.

La entrada a este tipo de recomendadores consistirá en una tabla de puntuaciones de los usuarios sobre los elementos y, por tanto, no necesitan información adicional ni de los usuarios ni de los elementos para poder hacer las recomendaciones.

Las salidas suelen ser:

- Una predicción numérica indicando en qué grado un determinado elemento le gusta al usuario objetivo.
- Un lista de elementos a recomendar para dicho usuario.

<div id='descarga' />

### Descarga de datos

In [1]:
!wget http://files.grouplens.org/datasets/movielens/ml-100k.zip
!mv ml-100k.zip ../data
!unzip ../data/ml-100k.zip -d ../data

--2022-01-11 18:20:14--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  2.85MB/s    in 1.6s    

2022-01-11 18:20:16 (2.85 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

Archive:  ../data/ml-100k.zip
   creating: ../data/ml-100k/
  inflating: ../data/ml-100k/allbut.pl  
  inflating: ../data/ml-100k/mku.sh  
  inflating: ../data/ml-100k/README  
  inflating: ../data/ml-100k/u.data  
  inflating: ../data/ml-100k/u.genre  
  inflating: ../data/ml-100k/u.info  
  inflating: ../data/ml-100k/u.item  
  inflating: ../data/ml-100k/u.occupation  
  inflating: ../data/ml-100k/u.user  
  inflating: ../data/ml-100k/u1.base  
  inflating: ../data/ml-100k/u1.test  


<div id='carga' />

### Carga de datos

In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

sc = SparkContext()
spark = SparkSession(sc)

data_ratings = sc.textFile("../data/ml-100k/u.data")
data_users = sc.textFile("../data/ml-100k/u.user")
data_items = sc.textFile("../data/ml-100k/u.item")

sc.version

'2.4.5'

<div id='limpieza' />

### Limpieza de datos

Tras importar los datos en `RDD` pasamos ahora a crear un dataframe con las variables que nos interesan. Podemos consultar la estructura de nuestros datos así como el significado de las variables en el siguiente [enlace](http://files.grouplens.org/datasets/movielens/ml-100k-README.txt).

In [3]:
from pyspark.sql import Row

rdd_ratings = data_ratings.map(lambda x: x.split("\t"))
rdd_ratings = rdd_ratings.map(lambda x: Row(user_id=int(x[0]), item_id=int(x[1]), 
                                            rating=int(x[2]), timestamp=int(x[3])))
df_ratings = spark.createDataFrame(rdd_ratings)

rdd_users = data_users.map(lambda x: x.split("|"))
rdd_users = rdd_users.map(lambda x: Row(user_id=int(x[0]), age=int(x[1]), gender=x[2], 
                                        occupation=x[3], zip_code=x[4]))
df_users = spark.createDataFrame(rdd_users)

rdd_items = data_items.map(lambda x: x.split("|"))
rdd_items = rdd_items.map(lambda x: Row(item_id=int(x[0]), item_title=x[1], release_date=x[2], video_release_date=x[3], imdb_url=x[4], 
                                        genre_unknown=int(x[5]), genre_action=int(x[6]), genre_adventure=int(x[7]), genre_animation=int(x[8]), 
                                        genre_childrens=int(x[9]), genre_comedy=int(x[10]), genre_crime=int(x[11]), genre_documentary=int(x[12]), 
                                        genre_drama=int(x[13]), genre_fantasy=int(x[14]), genre_noir=int(x[15]), genre_horror=int(x[16]), 
                                        genre_musical=int(x[17]), genre_mistery=int(x[18]), genre_romance=int(x[19]), genre_scifi=int(x[20]), 
                                        genre_thriller=int(x[21]), genre_war=int(x[22]), genre_western=int(x[23])))
df_items = spark.createDataFrame(rdd_items)

df = df_ratings.join(df_users, df_ratings.user_id == df_users.user_id, "left").select(df_ratings.user_id, "item_id", "rating", "age", "gender", "occupation")
df = df.join(df_items, df.item_id == df_items.item_id, "left")

df.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- item_id: long (nullable = true)
 |-- rating: long (nullable = true)
 |-- age: long (nullable = true)
 |-- gender: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- genre_action: long (nullable = true)
 |-- genre_adventure: long (nullable = true)
 |-- genre_animation: long (nullable = true)
 |-- genre_childrens: long (nullable = true)
 |-- genre_comedy: long (nullable = true)
 |-- genre_crime: long (nullable = true)
 |-- genre_documentary: long (nullable = true)
 |-- genre_drama: long (nullable = true)
 |-- genre_fantasy: long (nullable = true)
 |-- genre_horror: long (nullable = true)
 |-- genre_mistery: long (nullable = true)
 |-- genre_musical: long (nullable = true)
 |-- genre_noir: long (nullable = true)
 |-- genre_romance: long (nullable = true)
 |-- genre_scifi: long (nullable = true)
 |-- genre_thriller: long (nullable = true)
 |-- genre_unknown: long (nullable = true)
 |-- genre_war: long (nullable = true)
 |-

In [4]:
df = df.drop(df_items.item_id)
df.toPandas()

,user_id,item_id,rating,age,gender,occupation,genre_action,genre_adventure,genre_animation,genre_childrens,...,genre_romance,genre_scifi,genre_thriller,genre_unknown,genre_war,genre_western,imdb_url,item_title,release_date,video_release_date
0,474,26,4,51,M,executive,0,0,0,0,...,0,0,0,0,0,0,http://us.imdb.com/M/title-exact?Brothers%20Mc...,"Brothers McMullen, The (1995)",01-Jan-1995,
1,222,26,3,29,M,programmer,0,0,0,0,...,0,0,0,0,0,0,http://us.imdb.com/M/title-exact?Brothers%20Mc...,"Brothers McMullen, The (1995)",01-Jan-1995,
2,270,26,5,18,F,student,0,0,0,0,...,0,0,0,0,0,0,http://us.imdb.com/M/title-exact?Brothers%20Mc...,"Brothers McMullen, The (1995)",01-Jan-1995,
3,293,26,3,24,M,writer,0,0,0,0,...,0,0,0,0,0,0,http://us.imdb.com/M/title-exact?Brothers%20Mc...,"Brothers McMullen, The (1995)",01-Jan-1995,
4,243,26,3,33,M,educator,0,0,0,0,...,0,0,0,0,0,0,http://us.imdb.com/M/title-exact?Brothers%20Mc...,"Brothers McMullen, The (1995)",01-Jan-1995,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,907,1326,4,25,F,other,0,0,0,0,...,0,0,0,0,0,0,http://us.imdb.com/M/title-exact?Boys%20(1996),Boys (1996),10-May-1996,
99996,859,1326,4,18,F,other,0,0,0,0,...,0,0,0,0,0,0,http://us.imdb.com/M/title-exact?Boys%20(1996),Boys (1996),10-May-1996,
99997,500,1326,4,28,M,administrator,0,0,0,0,...,0,0,0,0,0,0,http://us.imdb.com/M/title-exact?Boys%20(1996),Boys (1996),10-May-1996,
99998,199,1326,3,30,M,writer,0,0,0,0,...,0,0,0,0,0,0,http://us.imdb.com/M/title-exact?Boys%20(1996),Boys (1996),10-May-1996,


<div id='analisis' />

### Análisis descriptivo

Veamos el top 10 de películas con mayor número de votos:

In [5]:
import pyspark.sql.functions as F
df.groupBy("item_title").count().sort(F.col("count").desc()).show(10)

+--------------------+-----+
|          item_title|count|
+--------------------+-----+
|    Star Wars (1977)|  583|
|      Contact (1997)|  509|
|        Fargo (1996)|  508|
|Return of the Jed...|  507|
|    Liar Liar (1997)|  485|
|English Patient, ...|  481|
|       Scream (1996)|  478|
|    Toy Story (1995)|  452|
|Air Force One (1997)|  431|
|Independence Day ...|  429|
+--------------------+-----+
only showing top 10 rows



Las valoraciones (`rating`) tienen valores comprendido en el intervalo [1, 5]. A continuación mostramos algunas de las películas mejor valoradas:

In [6]:
df.groupBy("item_title").agg(F.avg("rating").alias("avg_rating")).sort(F.col("avg_rating").desc()).show(10)

+--------------------+----------+
|          item_title|avg_rating|
+--------------------+----------+
|Santa with Muscle...|       5.0|
|Great Day in Harl...|       5.0|
|They Made Me a Cr...|       5.0|
|Entertaining Ange...|       5.0|
|Marlene Dietrich:...|       5.0|
|     Star Kid (1997)|       5.0|
|Aiqing wansui (1994)|       5.0|
|Saint of Fort Was...|       5.0|
|Someone Else's Am...|       5.0|
|  Prefontaine (1997)|       5.0|
+--------------------+----------+
only showing top 10 rows



Podemos ver un mayor número de votaciones realizadas por hombres (género masculino `M`):

In [7]:
df.groupBy("gender").count().sort(F.col("count").desc()).show()

+------+-----+
|gender|count|
+------+-----+
|     M|74260|
|     F|25740|
+------+-----+



In [8]:
df.groupBy("gender").agg(F.sum("rating").alias("total_rating")).sort(F.col("total_rating").desc()).show()

+------+------------+
|gender|total_rating|
+------+------------+
|     M|      262085|
|     F|       90901|
+------+------------+



In [9]:
df.groupBy("gender").agg(F.avg("rating").alias("avg_rating")).sort(F.col("avg_rating").desc()).show()

+------+------------------+
|gender|        avg_rating|
+------+------------------+
|     F|3.5315073815073816|
|     M|3.5292889846485322|
+------+------------------+



Si agrupamos por ocupación (`occupation`):

In [10]:
df.groupBy("occupation").count().sort(F.col("count").desc()).show()

+-------------+-----+
|   occupation|count|
+-------------+-----+
|      student|21957|
|        other|10663|
|     educator| 9442|
|     engineer| 8175|
|   programmer| 7801|
|administrator| 7479|
|       writer| 5536|
|    librarian| 5273|
|   technician| 3506|
|    executive| 3403|
|   healthcare| 2804|
|       artist| 2308|
|entertainment| 2095|
|    scientist| 2058|
|    marketing| 1950|
|      retired| 1609|
|       lawyer| 1345|
|         none|  901|
|     salesman|  856|
|       doctor|  540|
+-------------+-----+
only showing top 20 rows



In [11]:
df.groupBy("occupation").agg(F.sum("rating").alias("total_rating")).sort(F.col("total_rating").desc()).show()

+-------------+------------+
|   occupation|total_rating|
+-------------+------------+
|      student|       77182|
|        other|       37879|
|     educator|       34658|
|     engineer|       28951|
|   programmer|       27836|
|administrator|       27191|
|    librarian|       18776|
|       writer|       18688|
|   technician|       12384|
|    executive|       11397|
|       artist|        8432|
|   healthcare|        8121|
|    scientist|        7432|
|entertainment|        7209|
|    marketing|        6797|
|      retired|        5578|
|       lawyer|        5024|
|         none|        3405|
|     salesman|        3067|
|       doctor|        1992|
+-------------+------------+
only showing top 20 rows



In [12]:
df.groupBy("occupation").agg(F.avg("rating").alias("avg_rating")).sort(F.col("avg_rating").desc()).show()

+-------------+------------------+
|   occupation|        avg_rating|
+-------------+------------------+
|         none| 3.779134295227525|
|       lawyer|3.7353159851301116|
|       doctor| 3.688888888888889|
|     educator|3.6706206312221985|
|       artist| 3.653379549393414|
|administrator|3.6356464768017114|
|    scientist| 3.611273080660836|
|     salesman| 3.582943925233645|
|   programmer|3.5682604794257147|
|    librarian| 3.560781338896264|
|        other|3.5523773797242804|
|     engineer| 3.541406727828746|
|   technician|3.5322304620650313|
|      student|3.5151432345038027|
|    marketing|3.4856410256410255|
|      retired|3.4667495338719703|
|entertainment|3.4410501193317424|
|       writer|3.3757225433526012|
|    executive|3.3491037320011756|
|    homemaker| 3.301003344481605|
+-------------+------------------+
only showing top 20 rows



Finalmente eliminamos las variables que no utilizaremos en nuestro modelo.

In [13]:
df = df.select("user_id", "item_id", "rating")

In [14]:
del data_items, data_ratings, data_users, rdd_items, rdd_ratings, rdd_users

In [15]:
who_ls

['F',
 'Row',
 'SparkContext',
 'SparkSession',
 'df',
 'df_items',
 'df_ratings',
 'df_users',
 'sc',
 'spark']

<div id='model' />

### Modelo y ajuste de parámetros

El proceso de selección del modelo se realizará a través del análisis de validación cruzada (**cross-validation**) con ajuste automático de hiperparámetros. Este ajuste se hace definiendo los posibles valores de los hiperparámetros del modelo y ejecutando una búsqueda en rejilla (**grid-search**). Los hiperparámetros del modelo ALS son:

- rank: cantidad de factores latentes en el modelo (4, 8 y 12 como valores seleccionados)
- maxIter: número máximo de iteraciones (5).
- regParam: parámetro de regularización (0.1, 0.05 y 0.01 como valores seleccionados)

Para realizar una comparativa entre los modelos obtenidos con el proceso anterior se establece como método de evaluación el cálculo del error cuadrático medio (**RMSE**) ya que se usa frecuentemente como principal métrica de evaluación en problemas de regresión. RMSE compara los valores predichos del conjunto de entrenamiento con los valores reales presentes en el conjunto de validación. Al agregar el error absoluto de las diferencias y tomar el promedio de esos valores obtenemos una medida del error del modelo. Cuanto menor es el error, mejor es la capacidad de pronóstico de ese modelo según el criterio RMSE. También se obtendrán otras métricas del error como **MSE, R2 y MAE**.

In [16]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import time

start_time = time.time()

train, test = df.randomSplit([0.8, 0.2])
train = train.withColumnRenamed("rating", "label")

# Modelo ALS
# Parámetro coldStartStrategy="drop" para asegurar que no obtenemos NaN en métricas de evaluación
als = ALS(maxIter=5, 
          userCol="user_id", 
          itemCol="item_id", 
          ratingCol="label", 
          seed=1, 
          coldStartStrategy="drop", 
          nonnegative=True)

# Parámetros grid
# rank -> factores latentes en el modelo
paramGrid = ParamGridBuilder().addGrid(als.regParam, [0.1, 0.05, 0.01]).addGrid(als.rank, [4, 8, 12]).build()

# Cross-Validator
crossval = CrossValidator(estimator=als,
                          estimatorParamMaps=paramGrid,
                          evaluator=RegressionEvaluator(metricName="rmse"),
                          numFolds=10)

cvModel = crossval.fit(train)


# Selcción del mejor modelo
best_als_model = cvModel.bestModel
print("Best number of latent factors (rank parameter): " + str(best_als_model.rank))
print("Best value of regularization factor: " + str(best_als_model._java_obj.parent().getRegParam()))
print("Best max iterations: " + str(best_als_model._java_obj.parent().getMaxIter()))


# Evaluación del modelo
pred_test = best_als_model.transform(test)
#evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
#rmse = evaluator.evaluate(pred_test)
#print("RMSE: " + str(rmse))

ev = RegressionEvaluator(labelCol = "rating")
print("RMSE test: {0}".format(ev.evaluate(pred_test, {ev.metricName: "rmse"})))
print("MSE test: {0}".format(ev.evaluate(pred_test, {ev.metricName: "mse"})))
print("MAE test: {0}".format(ev.evaluate(pred_test, {ev.metricName: "mae"})))
print("R2 test: {0}".format(ev.evaluate(pred_test, {ev.metricName: "r2"})))

print("Tiempo de entrenamiento: ", (time.time() - start_time) / 60, "minutos")

Best number of latent factors (rank parameter): 12
Best value of regularization factor: 0.1
Best max iterations: 5
RMSE test: 0.9253131493349755
MSE test: 0.8562044243322108
MAE test: 0.7396551369445475
R2 test: 0.3162887571091789
Tiempo de entrenamiento:  7.29392474492391 minutos


<div id='kfold' />

#### K-fold

Para obtener una evaluación más precisa del sobreajuste (**overfitting**) del modelo y del error real (en test), una buena práctica es, una vez que se ha seleccionado el modelo ALS con el mejor ajuste de hiperparámetros (proceso de selección del modelo realizado mediante validación cruzada con búsqueda en rejilla), se evalua el modelo ajustado para distintos conjuntos de entrenamiento y test seleccionados aleatoriamente a través de múltiples **K-fold** y, finalmente, se promedian los resultados de las distintas métricas de evaluación calculadas en cada K-fold. Este proceso nos proporcionará una evaluación más precisa de nuestro motor de recomendación como predictor de ratings ante nuevos datos de entrada nunca antes vistos por el modelo.

In [17]:
def overfitting_evaluation(predictions):
    predictions = predictions.na.drop()
    # RMSE
    rmse_evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
    rmse = rmse_evaluator.evaluate(predictions)
    print("Root Mean Square Error (RMSE) = " + str(rmse))
    # MSE
    mse_evaluator = RegressionEvaluator(metricName="mse", labelCol="rating", predictionCol="prediction")
    mse = mse_evaluator.evaluate(predictions)
    print("Mean Square Error (MSE) = " + str(mse))
    # MAE
    mae_evaluator = RegressionEvaluator(metricName="mae", labelCol="rating", predictionCol="prediction")
    mae = mae_evaluator.evaluate(predictions)
    print("Mean Absolute Error (MAE) = " + str(mae))
    # R2
    r2_evaluator = RegressionEvaluator(metricName="r2", labelCol="rating", predictionCol="prediction")
    r2 = r2_evaluator.evaluate(predictions)
    print("R2 metric = " + str(r2))
    return [rmse, mse, r2, mae]

def kfold_test_eval(df, it, regPar, rank, Kfolds=5):
    rmse_evaluations = []
    mse_evaluations = []
    r2_evaluations = []
    mae_evaluations = []
    
    for k in range(0, Kfolds):  
        train, test = df.randomSplit([0.8, 0.2])
        train = train.withColumnRenamed("rating", "label")
        tunned_als = als = ALS(userCol="user_id", 
                               itemCol="item_id", 
                               ratingCol="label", 
                               coldStartStrategy="drop", 
                               maxIter=it, 
                               regParam=regPar, 
                               rank=rank)
        model = tunned_als.fit(train)
        predictions = model.transform(test)
        print("----> K-fold: " + str(k + 1))
        k_test_eval = overfitting_evaluation(predictions)
        rmse_evaluations.append(k_test_eval[0])
        mse_evaluations.append(k_test_eval[1])
        r2_evaluations.append(k_test_eval[2])
        mae_evaluations.append(k_test_eval[3])
        
    average_rmse = sum(rmse_evaluations)/float(len(rmse_evaluations))
    average_mse = sum(mse_evaluations)/float(len(mse_evaluations))
    average_r2 = sum(r2_evaluations)/float(len(r2_evaluations))
    average_mae = sum(mae_evaluations)/float(len(mae_evaluations))
    average_metrics = {"RMSE": average_rmse, "MSE": average_mse, "MAE": average_mae, "R2": average_r2}
    
    return average_metrics

In [18]:
start_time = time.time()
kfold_test_eval(df, it=5, regPar=0.1, rank=12)
print(" ")
print("Tiempo de ejecución k-fold: ", (time.time() - start_time) / 60, "minutos")

----> K-fold: 1
Root Mean Square Error (RMSE) = 0.9281340319558975
Mean Square Error (MSE) = 0.8614327812747109
Mean Absolute Error (MAE) = 0.738484669080392
R2 metric = 0.32393874295462544
----> K-fold: 2
Root Mean Square Error (RMSE) = 0.9318297056278995
Mean Square Error (MSE) = 0.8683066002905778
Mean Absolute Error (MAE) = 0.7407927340046081
R2 metric = 0.32618615308434473
----> K-fold: 3
Root Mean Square Error (RMSE) = 0.9226281800226845
Mean Square Error (MSE) = 0.8512427585719713
Mean Absolute Error (MAE) = 0.7333176278626808
R2 metric = 0.3150196018780358
----> K-fold: 4
Root Mean Square Error (RMSE) = 0.9276696963460531
Mean Square Error (MSE) = 0.8605710655187784
Mean Absolute Error (MAE) = 0.7368962762011714
R2 metric = 0.3164184526973567
----> K-fold: 5
Root Mean Square Error (RMSE) = 0.9391476817935454
Mean Square Error (MSE) = 0.8819983682181902
Mean Absolute Error (MAE) = 0.745802440046153
R2 metric = 0.31347863186101976
 
Tiempo de ejecución k-fold:  2.261584301789602 

<div id='resultado' />

### Resultado del modelo

Comprobaremos algunas predicciones obtenidas del modelo.

In [19]:
print("Top 3 recomendaciones de películas para cada usuario:")
userRecs = best_als_model.recommendForAllUsers(3)
userRecs.show(5, truncate=False)

Top 3 recomendaciones de películas para cada usuario:
+-------+--------------------------------------------------------+
|user_id|recommendations                                         |
+-------+--------------------------------------------------------+
|471    |[[538, 5.170471], [887, 5.0484133], [913, 4.9339857]]   |
|463    |[[1643, 4.522373], [1589, 4.4196224], [1449, 4.26085]]  |
|833    |[[1643, 4.8221817], [320, 4.2892814], [1368, 4.162993]] |
|496    |[[1643, 4.76684], [853, 4.1428933], [1467, 4.023804]]   |
|148    |[[1463, 5.5031233], [1643, 5.1378045], [814, 4.9099703]]|
+-------+--------------------------------------------------------+
only showing top 5 rows



In [20]:
print("Top 3 recomendaciones de usuario para cada película:")
movieRecs = best_als_model.recommendForAllItems(3)
movieRecs.show(5, truncate=False)

Top 3 recomendaciones de usuario para cada película:
+-------+------------------------------------------------------+
|item_id|recommendations                                       |
+-------+------------------------------------------------------+
|1580   |[[366, 1.5022786], [688, 1.4889176], [801, 1.4198372]]|
|471    |[[849, 4.703801], [939, 4.6878614], [810, 4.664942]]  |
|1591   |[[928, 5.8869095], [857, 5.501318], [519, 5.044736]]  |
|1342   |[[310, 3.9530053], [239, 3.876279], [4, 3.8565137]]   |
|463    |[[928, 4.973973], [310, 4.8433633], [909, 4.6595435]] |
+-------+------------------------------------------------------+
only showing top 5 rows



In [21]:
print("Top 3 películas recomendadas para un conjunto específico de usuarios (5):")
users = df.select(als.getUserCol()).distinct().limit(5)
userSubsetRecs = best_als_model.recommendForUserSubset(users, 3)
userSubsetRecs.show(truncate=False)

Top 3 películas recomendadas para un conjunto específico de usuarios (5):
+-------+--------------------------------------------------------+
|user_id|recommendations                                         |
+-------+--------------------------------------------------------+
|65     |[[1643, 5.295609], [113, 4.6787124], [318, 4.640968]]   |
|26     |[[1643, 4.757546], [113, 4.230652], [1467, 4.0680833]]  |
|474    |[[1643, 5.825402], [1449, 5.039047], [1467, 4.938492]]  |
|29     |[[1643, 5.6532516], [1449, 4.9601517], [1639, 4.815363]]|
|541    |[[1643, 5.130844], [814, 4.8571224], [1639, 4.6165576]] |
+-------+--------------------------------------------------------+



In [22]:
print("Top 3 usuarios recomendados para un conjunto específico de películas (5):")
movies = df.select(als.getItemCol()).distinct().limit(5)
movieSubSetRecs = best_als_model.recommendForItemSubset(movies, 3)
movieSubSetRecs.show(truncate=False)

Top 3 usuarios recomendados para un conjunto específico de películas (5):
+-------+------------------------------------------------------+
|item_id|recommendations                                       |
+-------+------------------------------------------------------+
|26     |[[688, 4.203037], [274, 4.0799584], [34, 4.0450025]]  |
|474    |[[928, 5.109949], [686, 5.0015645], [118, 4.961344]]  |
|1677   |[[118, 3.6793573], [34, 3.553275], [928, 3.5344198]]  |
|29     |[[127, 4.9205], [636, 4.260722], [261, 4.1712966]]    |
|964    |[[219, 4.6748614], [122, 4.3550925], [565, 4.3343263]]|
+-------+------------------------------------------------------+



Otra forma de extraer información de recomendaciones es mediante el método `take`. Veamos un ejemplo para las recomendaciones de usuarios para un conjunto de películas:

In [23]:
movieSubSetRecs.take(2)

[Row(item_id=26, recommendations=[Row(user_id=688, rating=4.203036785125732), Row(user_id=274, rating=4.079958438873291), Row(user_id=34, rating=4.045002460479736)]),
 Row(item_id=474, recommendations=[Row(user_id=928, rating=5.109949111938477), Row(user_id=686, rating=5.0015645027160645), Row(user_id=118, rating=4.961343765258789)])]

O bien mediante una tabla que muestre los usuarios recomendados (sin rating) además del título de la película:

In [24]:
#movieSubSetRecs.join(df_items, movieSubSetRecs.item_id == df_items.item_id, "left").select(df_items.item_id, "item_title", "recommendations").show()
movieSubSetRecs.join(df_items, movieSubSetRecs.item_id == df_items.item_id, "left").select(df_items.item_id, "item_title", movieSubSetRecs.recommendations.user_id).show()

+-------+--------------------+-----------------------+
|item_id|          item_title|recommendations.user_id|
+-------+--------------------+-----------------------+
|     26|Brothers McMullen...|         [688, 274, 34]|
|     29|Batman Forever (1...|        [127, 636, 261]|
|    474|Dr. Strangelove o...|        [928, 686, 118]|
|    964|Month by the Lake...|        [219, 122, 565]|
|   1677|Sweet Nothing (1995)|         [118, 34, 928]|
+-------+--------------------+-----------------------+

